# Pipeline

In [27]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [28]:
cancer = datasets.load_breast_cancer()
df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)

# 0: malignant (bad), 1: benign (Not bad)

df["target"] = cancer.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


## Split Data

In [29]:
X = df.drop(columns=["target"])
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")

Train: (455, 30)
Test: (114, 30)


## Pipeline

In [30]:
pipe_svm = Pipeline([('scale', StandardScaler()),
                     ('svm', SVC(kernel="linear"))])
pipe_svm.fit(X_train, y_train)

Pipeline(steps=[('scale', StandardScaler()), ('svm', SVC(kernel='linear'))])

In [31]:
hyperparams = {"random_state": 42,
               "max_iter": 1000,
               "hidden_layer_sizes": (20, 10,),
               "n_iter_no_change": 100,
               "early_stopping": True,
               "verbose": False}

features_to_scale = ["mean radius", "mean texture", "mean perimeter",
                     "mean area", "radius error","texture error",
                     "perimeter error", "area error", "worst radius",
                     "worst texture", "worst perimeter", "worst area",
                     "worst compactness", "worst concavity"]

scaling_pipe = Pipeline([('scale', MinMaxScaler())])
scaling_transformer = ColumnTransformer(transformers = [('minmax', scaling_pipe, features_to_scale)],
                                        remainder='passthrough')

pipe_nn = Pipeline([('scale', scaling_transformer),
                    ('nn', MLPClassifier(**hyperparams))])
pipe_nn.fit(X_train, y_train)

Pipeline(steps=[('scale',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('minmax',
                                                  Pipeline(steps=[('scale',
                                                                   MinMaxScaler())]),
                                                  ['mean radius',
                                                   'mean texture',
                                                   'mean perimeter',
                                                   'mean area', 'radius error',
                                                   'texture error',
                                                   'perimeter error',
                                                   'area error', 'worst radius',
                                                   'worst texture',
                                                   'worst perimeter',
                                                   'worst area',
                                                   'worst compactness',
                                                   'worst concavity'])])),
                ('nn',
                 MLPClassifier(early_stopping=True, hidden_layer_sizes=(20, 10),
                               max_iter=1000, n_iter_no_change=100,
                               random_state=42))])

## Evaluation

In [34]:
pipe_svm.score(X_test, y_test)

0.956140350877193

In [35]:
pipe_nn.score(X_test, y_test)

0.9736842105263158